Installing the necessary libraries

In [ ]:
!pip install firecrawl-py
!pip install mailjet_rest
!pip install llama_index
!pip install llama-index llama-index-llms-groq groq

In [ ]:
import requests
import time
import re
from firecrawl import FirecrawlApp
from requests.exceptions import HTTPError
from llama_index.llms.groq import Groq
import textwrap
from mailjet_rest import Client
import os

Run this cell
The user has to give input about search query, recipient email address and their name.

In [ ]:


# Initialize the Groq LLM
llm_groq = Groq(model="llama3-70b-8192", api_key="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") 

# Function to fetch search results
def fetch_search_results(query):
    api_key = '6e7a28dfb535d1bd93af90c03765a36c43da33cd61154c59361c3bfa0f176db9'  
    url = f"https://serpapi.com/search?api_key={api_key}&q={query}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        results = response.json()
        return results.get('organic_results', [])[:2]  
    else:
        print("Error fetching results:", response.status_code)
        return []

# Function to clean Website content
def clean_wikipedia_content(content):
    if not isinstance(content, str):
        content = str(content)
        
    cleaned_content = re.sub(r'\[.*?\]', '', content)  
    cleaned_content = re.sub(r'https?://\S+', '', cleaned_content)  
    cleaned_content = re.sub(r'!\(data:image/\S+;base64,[^\)]+\)', '', cleaned_content)  
    cleaned_content = re.sub(r'::+', ' ', cleaned_content)  
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content)  
    
    return cleaned_content.strip()

# Function to split content into chunks
def split_content_into_chunks(content, chunk_size=15000):
    return textwrap.wrap(content, chunk_size)

# Function to summarize each chunk with Groq
def summarize_with_groq(text):
    prompt = f"""
    Generate a summary of the text which explains everything in 200 words at maximum. Do not exceed this limit.
    
    Text: {text}
    """
    
    response = llm_groq.complete(prompt)
    return response.text  

# Function to scrape content from URLs with rate limit handling
def scrape_urls(urls, firecrawl):
    all_summaries = []
    
    for url in urls:
        while True:
            try:
                print(f"Scraping {url}...")
                
                # Scrape the URL and get the content
                page_content = firecrawl.scrape_url(url=url, params={"onlyMainContent": True})
                
                # Ensure the content is a string
                if not isinstance(page_content, str):
                    page_content = str(page_content)
                
                # Clean the content
                cleaned_content = clean_wikipedia_content(page_content)
                
                # Split the cleaned content into chunks
                chunks = split_content_into_chunks(cleaned_content)
                
                # Summarize each chunk using Groq
                chunk_summaries = [summarize_with_groq(chunk) for chunk in chunks]
                
                # Combine chunk summaries
                combined_summary = " ".join(chunk_summaries)
                
                # Add this URL's combined summary to the list of all summaries
                all_summaries.append(combined_summary)
                
                # Print the URL's combined summary
                print(f"Summary from {url}:")
                print(combined_summary)
                print("\n" + "="*80 + "\n")  
                break  
            
            except HTTPError as e:
                if e.response.status_code == 429:
                    # Handle rate limit exceeded error
                    reset_time = e.response.headers.get('Retry-After', 45) 
                    print(f"Rate limit exceeded. Pausing for {reset_time} seconds...")
                    time.sleep(int(reset_time))  
                else:
                    print(f"An error occurred: {e}")
                    break  

    # Combine all URL summaries
    large_text = " ".join(all_summaries)
    
    # Create the prompt for the final summarization
    final_prompt = f"""
    Based on the following text, generate:
    1. A title
    2. An introduction
    3. Five top subtopic titles
    4. A short summary
    5. A conclusion

    Text: {large_text}
    """
    
    # Generate the final summary with Groq
    final_summary = llm_groq.complete(final_prompt)
    
    # Print the final summary
    print("Final Summary of All URLs:")
    print(final_summary)

    return final_summary.text  

# Function to send an email using Mailjet
def send_email_mailjet(subject, body, recipient, rec_name):
    api_key = 'fd5ec7040540bf0b2ffe1f6d5bcc49aa'
    api_secret = "b0a5c161ba03369f3ed18c7ab530b52e"
    
    mailjet = Client(auth=(api_key, api_secret), version='v3.1')
    
    data = {
        'Messages': [
            {
                'From': {
                    'Email': 'nrj.eea@gmail.com',
                    'Name': 'Neeraj'
                },
                'To': [
                    {
                        'Email': recipient,
                        'Name': rec_name
                    }
                ],
                'Subject': subject,
                'TextPart': body,
                'HTMLPart': f"<p>{body}</p>" 
            }
        ]
    }
    
    result = mailjet.send.create(data=data)
    print(result.status_code)
    print(result.json())

# Initialize the Firecrawl application with your API key
api_key = "fc-0d7421cf1f3c467aa17269df26a307e8"  
firecrawl = FirecrawlApp(api_key=api_key)

# Step 1: Fetch search results
query = input("Enter your search query: ")
results = fetch_search_results(query)

# Step 2: Extract URLs from the search results
urls = [result.get('link') for result in results if result.get('link')]

# Step 3: Scrape content from the URLs
final_summary = scrape_urls(urls, firecrawl)

# Step 4: Send the final summary via email
recipient_email = input("Enter recipient email address: ")
rec_name = input("Enter the name: ")
send_email_mailjet("Final Summary of Search Results", final_summary, recipient_email, rec_name)